In [ ]:
import os
import json

def json_to_yolo(json_data, image_width, image_height):
    yolo_data = []

    for annotation in json_data['annotations']:
        class_index = annotation['category_id']
        x, y, width, height = annotation['bbox']

        # Normalize bounding box coordinates
        normalized_x = x / image_width
        normalized_y = y / image_height
        normalized_width = width / image_width
        normalized_height = height / image_height

        yolo_line = f"{class_index} {normalized_x} {normalized_y} {normalized_width} {normalized_height}"

        # Normalize keypoint coordinates
        keypoints = annotation['keypoints']
        for i in range(0, len(keypoints), 3):
            px, py, visibility = keypoints[i], keypoints[i+1], keypoints[i+2]

            normalized_px = px / image_width
            normalized_py = py / image_height

            yolo_line += f" {normalized_px} {normalized_py} {visibility}"

        yolo_data.append(yolo_line)

    return yolo_data

def process_json_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            json_file_path = os.path.join(folder_path, filename)

            with open(json_file_path, 'r') as json_file:
                json_data = json.load(json_file)

                # Modify image_width and image_height based on your dataset
                image_width = json_data['images'][0]['width']
                image_height = json_data['images'][0]['height']

                yolo_data = json_to_yolo(json_data, image_width, image_height)

                # Save YOLO-formatted data to a file
                yolo_file_path = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}.txt")
                with open(yolo_file_path, 'w') as yolo_file:
                    for line in yolo_data:
                        yolo_file.write(line + '\n')

if __name__ == "__main__":
    folder_path = input("Enter the folder path containing JSON files:  ")
    process_json_files(folder_path)
